# Sky Coordinates of DC2 calexps and deepCoadds in Jupyter Hub

Arthur: Shu Liu

Questions we want to answer:

- Given a list of sky coordinates, how can we get the (tract, patch, filter) information of the deepCoadds that covering these positions in LSST Jupyter hub? 

- Given a list of sky coordinates, how can we get the (visit, detector, filter) information of the calexp that covering these positions in LSST Jupyter hub?

- For a coordinate which lies near the edge of the image, how can we check it is near the edge? 


#### 1. Given a list of sky coordinates, how can we get the (tract, patch, filter) information of the deepCoadds that covering these positions in LSST Jupyter hub?

In [1]:
from lsst.daf.persistence import Butler
import lsst.geom as geom

In [2]:
# Run 2.2i dataset
coadd_repo = '/global/cfs/projectdirs/lsst/production/DC2_ImSim/Run2.2i/desc_dm_drp/v19.0.0-v1/rerun/run2.2i-coadd-wfd-dr6-v1-grizy'
butler = Butler(coadd_repo)

In [3]:
skymap = butler.get('deepCoadd_skyMap')

In [4]:
# This function is modified from the get_coadd_id_for_ra_dec function from dm_utilities.py
def get_deepCoadd_id(skymap, ra, dec):
    
    radec = geom.SpherePoint(ra, dec, geom.degrees)
    # skymap.findTract finds the tract with the nearest center to the target
    # skymap.findAllTracts finds all tracts which include the target
    tract_info = skymap.findTract(radec)
    patch_info = tract_info.findPatch(radec)
    patch_idx = patch_info.getIndex()
    coadd_id = {'tract': tract_info.getId(), 
                'patch': f'{patch_idx[0]},{patch_idx[1]}' }
    return coadd_id

In [5]:
# example

ra = 57.082
dec = -31.1539

# expect tract=4639, patch=0,0
get_deepCoadd_id(skymap, ra, dec)

{'tract': 4639, 'patch': '0,0'}

`Question`: How can we get the filter information?

#### 2. Given a list of sky coordinates, how can we get the (visit, detector, filter) information of the calexp that covering these positions in LSST Jupyter hub? 

Suppose we have already get the coadd which includes the coordinate, we can search for calexps which overlap the coadd, and then check whether they include the coordinate.

In [6]:
import sqlite3
import pandas as pd

In [7]:
# connect to the calexp database
conn = sqlite3.connect('/global/cfs/projectdirs/lsst/production/DC2_ImSim/Run2.2i/desc_dm_drp/v19.0.0-v1/rerun/run2.2i-calexp-v1/tracts_mapping.sqlite3')

In [10]:
query = "select distinct(visit), filter, detector from overlaps where tract=4639 and filter='r' order by visit"
# use pandas to read the sql query
calexp_info = pd.read_sql_query(query, conn)

In [11]:
calexp_info.head()

,visit,filter,detector
0,181868,r,54
1,181868,r,55
2,181868,r,56
3,181868,r,57
4,181868,r,58


In [16]:
def in_image(ra, dec, wcs, bbox, offset):
    radec = geom.SpherePoint(ra, dec, geom.degrees)
    xy = wcs.skyToPixel(radec)
    # getX() and getY() return the physics coordinates, not the image coordinates
    phy_x, phy_y = xy.getX(), xy.getY()
    if bbox.beginX + offset <= phy_x and bbox.endX - offset >= phy_x and \
    bbox.beginY + offset <= phy_y and bbox.endY - offset >= phy_y:
        return True
    else:
        return False

In [17]:
calexp_repo = '/global/cfs/projectdirs/lsst/production/DC2_ImSim/Run2.2i/desc_dm_drp/v19.0.0-v1/rerun/run2.2i-calexp-v1'
calexp_butler = Butler(calexp_repo)

In [18]:
# example
calexp_id = {'visit': 181868, 'detector': 54, 'filter': 'r'}
calexp_exposure = calexp_butler.get('calexp',  calexp_id)
wcs = calexp_exposure.getWcs()
bbox = calexp_exposure.getBBox()

In [19]:
ra = 57.082
dec = -31.1539
in_image(ra, dec, wcs, bbox, offset=0)

False

In [20]:
ra = 57.105
dec = -31.239
in_image(ra, dec, wcs, bbox, offset=0)

True

`Question`: Loading the calexp exposure takes time, is there any other method for mapping skycoordinates to calexp images? 

#### 3. For a coordinate which lies near the edge of the image, how can we check it is near the edge?

`Answer` Get the corresponding exposure, convert skycoordinates to physics coordinates, and compare to the bbox of the exposure.

In [23]:
def cal_offset(ra, dec, wcs, bbox):
    radec = geom.SpherePoint(ra, dec, geom.degrees)
    xy = wcs.skyToPixel(radec)
    # getX() and getY() return the physics coordinates, not the image coordinates
    phy_x, phy_y = xy.getX(), xy.getY()
    dx_begin = phy_x - bbox.beginX
    dx_end =  bbox.endX - phy_x
    dy_begin = phy_y - bbox.beginY
    dy_end = bbox.endY - phy_y
    return {'dx_begin': dx_begin, 'dx_end': dx_end, 'dy_begin': dy_begin, 'dy_end': dy_end}
    

In [29]:
# Example
ra = 57.082
dec = -31.1539
cal_offset(ra, dec, wcs, bbox)
# it is not in the image, we expect to see at least one negative value

{'dx_begin': 2249.3351028976776,
 'dx_end': 1822.6648971023224,
 'dy_begin': 5120.428898633786,
 'dy_end': -1120.428898633786}

In [26]:
ra = 57.105
dec = -31.239
cal_offset(ra, dec, wcs, bbox)
# it is in the image, we expect all values be positive

{'dx_begin': 1842.9406496097865,
 'dx_end': 2229.0593503902137,
 'dy_begin': 3601.463703606275,
 'dy_end': 398.536296393725}

#### Other Questions.

- Given a calexp / deepCoadd exposure, how can we get the bounding sky coordinates?

For example, when we are searching sources using the GCRCatalogs, we can set a limit of ra and dec:

```
gc = GCRCatalogs.load_catalog('cosmoDC2_v1.1.4_image')
galaxy_data = gc.get_quantities(['ra', 'dec', 'mag_r', 'halo_id'],
                                filters=['mag_r > 20', 'mag_r < 22',
                                         'ra > 56.96', 'ra < 57.2',
                                        'dec > -31.26', 'dec < -31.05'])
```
If we want to select the sources in a given image, we need to know the bouding sky coordinates, is there any function can achieve this?

We can convert the pixel coordinates of the boundary to the sky coordinates, but it does not seem optimal.

```
radec = geom.SpherePoint(wcs.pixelToSky(x, y))
ra = radec.getRa().asDegrees()
dec = radec.getDec().asDegrees()
```


- Given a specific time range, how can we get calexp exposures taken within these days? How can we check the sky area they cover?